In [18]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from tifffile import imread, imwrite
import matplotlib.pyplot as plt
from skimage.transform import rotate
from skimage.morphology import disk, binary_opening, binary_dilation, label
from skimage.filters import threshold_otsu
from skimage.measure import centroid
from skimage.measure import regionprops, regionprops_table
import pickle

def tileOnCentroid(image, segmentation, labels, centroidX, centroidY, orientations, tileSize, polarity=True):
    """
    Cell-centered tiles of tileSize
    """
    tiles = {}
    tileWidth = int(tileSize/2)
    paddedImage = np.pad(image_,pad_width=[(tileWidth,tileWidth),(tileWidth,tileWidth)])
    paddedSeg = np.pad(segmentation,pad_width=[(tileWidth,tileWidth),(tileWidth,tileWidth)])
    for iX, iY, iLabel, iO in zip(centroidX,centroidY,labels, orientations):
        nX = int(iX) + tileWidth
        nY = int(iY) + tileWidth       
        iTile = paddedImage[nX-tileWidth:nX+tileWidth, nY-tileWidth:nY+tileWidth].copy()
        iSeg = paddedSeg[nX-tileWidth:nX+tileWidth, nY-tileWidth:nY+tileWidth].copy()
        if checkEdge(nX, nY, image_.shape, tileWidth)  == True:
            iTile[iSeg!=iLabel] = 0
            if np.sum(iTile) == 0:
                iTile[int(tileWidth/2),int(tileWidth/2)] = 1
            newTile = alignPolarityChannel(iTile.copy(),0)
            newTile /= (np.sum(newTile) + 1e-10) # divide pixel intensities by sum of pixel intensities
            tiles[iLabel] = newTile
    return tiles 


def alignPolarityChannel(image, threshold):
    """
    Aligns polarity of image (max projection)
    """
    imageProjection = image.copy()
    mask = imageProjection.copy()
    mask[mask>threshold] = 1
    angle = regionprops(mask)[0]['orientation']
    rotatedImage = rotate(image.copy(), -angle/2/3.14*360, resize=False, preserve_range=True)
    if np.sum(imageProjection[:,0:int(image.shape[1]/2)]) < np.sum(imageProjection[:,int(image.shape[1]/2):-1]):
        rotatedImage = np.flip(rotatedImage, axis=1)
    if np.sum(imageProjection[0:int(image.shape[1]/2),:]) < np.sum(imageProjection[int(image.shape[1]/2):-1,:]):
        rotatedImage = np.flip(rotatedImage, axis=0)
    return rotatedImage

def checkEdge(xCoord, yCoord, imageSize, tileWidth):
    """
    Check if the tile is overlapping the edge
    """
    xMin, xMax = (0, imageSize[0])
    yMin, yMax = (0, imageSize[1])
    if (xCoord - tileWidth <= tileWidth) | ((xCoord + tileWidth) > xMax + tileWidth):
        return False
    elif (yCoord - tileWidth <= tileWidth) | ((yCoord + tileWidth) > yMax + tileWidth):
        return False
    else:
        return True
    
def getCentroids(segmentationMask):
    """
    Return centroids of cells in segmentation mask
    """
    rps = pd.DataFrame(regionprops_table(segmentationMask.astype(np.uint64),properties=['label','centroid','orientation']))
    return (list(rps['label']),list(rps['centroid-0']),list(rps['centroid-1']), list(rps['orientation']))

parentPath = '/project/zunderlab/heussner/Projects/morph'

In [16]:
markers = ['DAPI','CD8','CD3','CD20','Ki67','CD68','PanCK','CD21','CD4','CD31','CD45RO','CD11c','HLA-DR']

In [8]:
image = imread(os.path.join(parentPath,'data/CODEX_LN/CODEX_LN_image.tif'))

In [9]:
segmentation = imread(os.path.join(parentPath,'data/CODEX_LN/CODEX_LN_mask.tif'))
labels, centroidX, centroidY, orientations = getCentroids(segmentation)

In [19]:
for i in range(len(markers)):
    image_ = image[i+2,:,:].copy()
    tiles = tileOnCentroid(image_, segmentation, labels, centroidX, centroidY, orientations, 32, polarity=True)
    with open(os.path.join(parentPath,'data/CODEX_LN/CODEX_LN_tiles/',f'{markers[i]}.pkl'),'wb') as handle:
        pickle.dump(tiles,handle)

In [62]:
names = imageTiles.keys()
saveDir = os.path.join(parentPath,'data/CODEX_LN/CODEX_LN_tiles/')
for iName in tqdm(names):
    trajectorySaveDir = os.path.join(saveDir,iName)
    with open(os.path.join(trajectorySaveDir,f'{iName}.pkl'),'wb') as handle:
        pickle.dump(imageTiles[iName],handle)

100%|██████████| 1/1 [00:32<00:00, 32.60s/it]
